In [129]:
import requests
import json

In [136]:
def Json_a_PlantUML(json_data):
    codigo_PlantUML = "@startuml\n"
    clases_definidas = set()  # Evita clases duplicadas

    def process_object(name, obj):
        if name in clases_definidas:
            return ""  # No repetir clases

        clases_definidas.add(name)
        class_PlantUML = f"class {name} {{\n"
        attributes = ""
        relationships = ""

        for key, value in obj.items():
            if isinstance(value, dict):  # Si es un objeto anidado
                class_name = key.capitalize()
                relationships += f"{name} --> {class_name} : tiene\n"
                relationships += process_object(class_name, value)  # Procesa subclase
            else:
                type_name = "String" if isinstance(value, str) else "Integer"
                attributes += f"  + {key}: {type_name}\n"

        class_PlantUML += attributes + "}\n\n"  # Cierre correcto de clase
        return class_PlantUML + relationships

    # Generar código PlantUML sin errores de llaves
    codigo_PlantUML += process_object("Root", json_data)
    codigo_PlantUML += "@enduml"

    return codigo_PlantUML


In [137]:
json_data = {
    "Usuario": {
        "nombre": "Juan",
        "edad": 25,
        "direccion": {
            "calle": "Calle 123",
            "ciudad": "Bogotá"
        }
    }
}

plantuml_code = Json_a_PlantUML(json_data)

with open("diagrama.puml", "w") as f:
    f.write(plantuml_code)


In [139]:
response = requests.post(
    "https://kroki.io/plantuml/svg",
    data=plantuml_code.encode("utf-8"),
    headers={"Content-Type": "text/plain"},
)

# Guardar la imagen generada
with open("diagrama.svg", "wb") as f:
    f.write(response.content)